In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score
import scipy
import scipy.stats as st
from scipy.special import logit

In [ ]:
import pandas as pd

data = pd.read_csv('data-001.csv')

#Удаление фичей, которые использоваться не будут
columns_to_remove = ['campaign_clicks', 'impressions']
data = data.drop(columns=columns_to_remove)

data.head()

,date_time,zone_id,banner_id,oaid_hash,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

def feature_engineering(data: pd.DataFrame) -> (pd.DataFrame, pd.Series, pd.DataFrame, pd.Series):

    # Фильтрация данных по времени и удаление выбросов в дате и g0
    data['date_time'] = pd.to_datetime(data['date_time'])
    data = data[data['date_time'] > '2021-09-20']
    data = data[(data['g0'] > 1e-6) & (data['g1'] > 1e-6)]

    #Оставляем только строки, где banner_id == banner_id_0
    data = data[data['banner_id'] == data['banner_id0']]
    data.drop(["banner_id0"], axis=1, inplace=True)

    # Добавление фичи "Время суток"
    data['time_of_day'] = pd.cut(data['date_time'].dt.hour,
                                 bins=[0, 6, 12, 18, 24],
                                 labels=['Night', 'Morning', 'Afternoon', 'Evening'])

    # Добавление фичи "День недели"
    data['day_of_week'] = data['date_time'].dt.day_name()


    # Разделение на обучающий и тестовый наборы и подготовка лейблов
    train_data = data[data['date_time'] < '2021-10-02']
    test_data = data[data['date_time'] >= '2021-10-02']
    columns_to_remove = ["clicks", "date_time", "banner_id1", "g0", "g1", "coeff_sum0",  "coeff_sum1"]
    X_train = train_data.drop(columns_to_remove, axis=1, inplace=False)
    X_test = test_data.drop(columns_to_remove, axis=1, inplace=False)
    y_train = train_data['clicks']
    y_test = test_data['clicks']

    # Сохранение данных для расчета \pi_0 и \pi_1
    pi_data = test_data[["g0", "g1", "coeff_sum0", "coeff_sum1"]]

    #One hot encoding
    encoder = OneHotEncoder(handle_unknown="ignore", sparse=True)
    X_train = encoder.fit_transform(X_train)
    X_test = encoder.transform(X_test)

    columns_to_remove = ["clicks", "date_time", "g0", "g1", "coeff_sum0",  "coeff_sum1"]
    test_data.drop(columns_to_remove, axis=1, inplace=True)

    # pi_0 pi_1
    test_data_pi0 = test_data.drop(["banner_id1"], axis=1, inplace=False)
    test_data_pi0 = encoder.transform(test_data_pi0)
    test_data["banner_id"] = test_data["banner_id1"]
    test_data_pi1 = test_data.drop(["banner_id1"], axis=1)
    test_data_pi1 = encoder.transform(test_data_pi1)

    return X_train, y_train, X_test, y_test, test_data_pi0, test_data_pi1, pi_data



In [ ]:
X_train, y_train, X_test, y_test, test_data_pi0, test_data_pi1, pi_data = feature_engineering(data.copy())

In [ ]:
#Проверка размерностей

print("Размерность X_train:", X_train.shape)
print("Размерность y_train:", y_train.size)
print("Размерность X_test:", X_test.shape)
print("Размерность y_test:", y_test.size)
print("Размерность test_data_pi0:", test_data_pi0.shape)
print("Размерность test_data_pi1:", test_data_pi1.shape)
print("Размерность pi_data:", pi_data.shape)

Размерность X_train: (12041809, 5029335)
Размерность y_train: 12041809
Размерность X_test: (1885668, 5029335)
Размерность y_test: 1885668
Размерность test_data_pi0: (1885668, 5029335)
Размерность test_data_pi1: (1885668, 5029335)
Размерность pi_data: (1885668, 4)


X_test_pi0 и X_test_pi1 - это подготовленные тестовые наборы данных для оценки политик.

pi_data - это DataFrame, содержащий статистические распределения (в данном случае, параметры нормальных распределений), которые используются для расчета вероятностей $\pi_0$ и $\pi_1$.  Содержит колонки g0, g1, coeff_sum0, и coeff_sum1, которые необходимы для оценки вероятности того, что одна нормальная величина больше другой

In [ ]:
#Проверим работу модели (параметры взяты из HW1)
def create_model(solver = 'liblinear', C = 0.001, penalty = 'l2'):
    lr = LogisticRegression(solver=solver, C=C, penalty=penalty)
    return lr

model = create_model()
model.fit(X_train, y_train)

LogisticRegression(C=0.001, solver='liblinear')

In [ ]:
# Оценка модели на тестовых данных
def evaluate_model(model, x, y_true):
    y_pred_prob = model.predict_proba(x)[:, 1]
    logloss = log_loss(y_true, y_pred_prob)
    auc = roc_auc_score(y_true, y_pred_prob)
    return logloss, auc

logloss, auc = evaluate_model(model, X_test, y_test)
print("Log Loss on test data:", logloss)
print("AUC on test data:", auc)

# Бейзлайн
mean_baseline = y_train.mean()
baseline_predictions = [mean_baseline] * len(y_test)
logloss_baseline = log_loss(y_test, baseline_predictions)
auc_baseline = roc_auc_score(y_test, baseline_predictions)
print("Log Loss baseline:", logloss_baseline)
print("AUC baseline:", auc_baseline)

Log Loss on test data: 0.13600427989811867
AUC on test data: 0.784741775377027
Log Loss baseline: 0.15751369478239677
AUC baseline: 0.5


1. Для расчета $\pi_0$:

Пусть $\xi_0 \sim N(\mu_0, \sigma_0^2) \text{ и } \xi_1 \sim N(\mu_1, \sigma_1^2)$.

Введем $\eta = \xi_1 - \xi_0 \sim N(\mu_1 - \mu_0, \sqrt{\sigma_1^2 + \sigma_0^2}) $.

Тогда $P(\xi_0 > \xi_1) = P(\eta < 0) = F_{\eta}(0)$ ,

где $ F_{\eta}(0) $— значение функции распределения для $ \eta$  в точке 0.

2. Для расчета $\pi_1$:

Используем логиты предсказанных моделью вероятностей:
$\text{logit}(\pi_0)$ и $\text{logit}(\pi_1)$.

Пусть $ \text{logit}(\pi_0) = \mu_0' $ и $ \text{logit}(\pi_1) = \mu_1'$.

 Тогда $ P(\text{logit}(\pi_0) > \text{logit}(\pi_1)) = P(\mu_1' - \mu_0' < 0) = F_{\eta'}(0)$,
   
где $\eta' = \mu_1' - \mu_0' \sim N(\mu_1' - \mu_0', \sqrt{\sigma_1^2 + \sigma_0^2})$ и $ F_{\eta`}(0)$ — значение функции распределения для $\eta$ в точке 0.

In [ ]:
# Функция для расчета вероятности того, что одна нормальная величина больше другой
def calculate_pi(coeff_sum0, coeff_sum1, g0, g1):
    diff = coeff_sum1 - coeff_sum0
    std_diff = np.sqrt(g0**2 + g1**2 + 1e-6)  # 1e-6 чтобы избежать дальше деления на ноль
    pi = st.norm.cdf(diff / std_diff)
    return pi

In [ ]:
# Использование функции для расчета pi_0
pi_0 = calculate_pi(pi_data.coeff_sum0, pi_data.coeff_sum1, pi_data.g0, pi_data.g1)


# Теперь считаем pi_1
# Получение предсказанных вероятностей модели
prob_pi0 = model.predict_proba(test_data_pi0)[:, 1]
prob_pi1 = model.predict_proba(test_data_pi1)[:, 1]

# Преобразование вероятностей в логиты (логарифм отношения шансов)
coeff_sum_pi0 = logit(prob_pi0)
coeff_sum_pi1 = logit(prob_pi1)

# Использование функции для расчета pi_1
pi_1 = calculate_pi(coeff_sum_pi0, coeff_sum_pi1, pi_data.g0, pi_data.g1)

Используя $\pi_0$ и $\pi_1$ посчитаем Clipped IPS по формуле из презентации:

$CIPS = \frac{1}{N} \sum_{i=1}^N r_i \min(\frac{\pi_1(a_i|x_i)}{\pi_0(a_i|x_i)}, \lambda)$

In [ ]:
# Расчет CIPS (Cumulative Incidence Probability Score)
LAMBDA = 10

cips = np.mean(y_test * np.clip(pi_1 / (pi_0 + 1e-6), 0, LAMBDA)) #1e-6 чтобы не делить на 0

print(f"CIPS = {cips:.9f}")

0.076750962
